# Tarea 4

## Introducción

En economía es muy famosa la función de producción de *Cobb-Douglas*, la cual veremos (eso espero) en el curso de **IIP314W Optimización aplicada a negocios**, puede revisarla en:

https://es.wikipedia.org/wiki/Funci%C3%B3n_de_producci%C3%B3n_de_Cobb-Douglas

Usted debe solucionar el siguiente problema, utilizando las rutinas de la librerias scipy.optimize que apliquen al problema, probando todos los metodos distintos que pueda, calculando tiempos de ejecución y diferencias relativas entre las soluciones. En este trabajo, no es necesario utilizar las rutinas propias que vimos en las dos clases anteriores. Entonces,

---


Consideremos una economía con **dos sectores productivos** (i = 1, 2).  
Cada sector i tiene una función de producción Cobb–Douglas:

$$Y_i = A_i \, K_i^{\alpha_i} \, L_i^{1-\alpha_i}$$

donde:
- $Y_i$: producción total del sector i (el valor monetario de todos los bienes producidos durante un año)
- $K_i$: capital asignado al sector i  
- $L_i$: trabajo asignado al sector i  
- $A_i$: productividad del sector i
- $\alpha_i$: elasticidad del capital en el sector i.

El capital y el trabajo totales son conocidos:

$$K_1 + K_2 = \bar K, \qquad L_1 + L_2 = \bar L.$$

### Condiciones de Equilibrio

En mercados competitivos, el costo de capital $r$ y el salario de trabajo $w$ se igualan a los productos marginales en cada sector:

$$r = \frac{\partial Y_i}{\partial K_i} \qquad i = 1,2.$$

$$w = \frac{\partial Y_i}{\partial L_i} \qquad i = 1,2.$$

Esto se debe a que en un mercado competitivo, todos los agentes (sectores, empresas) pueden mover capital y trabajo libremente. Si un sector pagara un costo mayor por el capital que otro, el capital migraría hacia ese sector hasta que los precios se igualen. Lo mismo con los salarios: si un sector pagara más por el trabajo, los trabajadores se moverían allí, hasta que los salarios se equilibren (supongamos que en Chile pasa eso).

Por eso, en equilibrio, existe un único *r* y un único *w* para toda la economía.

Esto da lugar a un sistema de 4 ecuaciones no lineales en las 4 incógnitas:


$$\begin{cases}
f_1(K_1,L_1,r,w) = r - A_1 \alpha_1 K_1^{\alpha_1-1} L_1^{1-\alpha_1} = 0, \\[6pt]
f_2(K_1,L_1,r,w) = r - A_2 \alpha_2 (K_2)^{\alpha_2-1} (L_2)^{1-\alpha_2} = 0, \\[6pt]
f_3(K_1,L_1,r,w) = w - A_1 (1-\alpha_1) K_1^{\alpha_1} L_1^{-\alpha_1} = 0, \\[6pt]
f_4(K_1,L_1,r,w) = w - A_2 (1-\alpha_2) (K_2)^{\alpha_2} (L_2)^{-\alpha_2} = 0.
\end{cases}
$$

Nota: Debemos reemplazar $K_2 = \bar K - K_1$ y $L_2 = \bar L - L_1$.

Solucione el sistema de ecuaciones con los siguientes datos:
$$
A_1 = 1.2, \quad A_2 = 0.9, \quad \alpha_1 = 0.3, \quad \alpha_2 = 0.6, 
\quad \bar K = 100, \quad \bar L = 50.
$$

Como solución inicial, empiece con alguna que le parezca razonable, si quiere puede usar la mia (abajo).

---

Realice un informe donde:

- Compare los resultados y tiempos de ejecución. Comente las diferencias de la variable *sol* de abajo.
- Modifique un poco la solución inicial o parametros, vea si sigue funcionando o empieza a funcionar.
- Investigue qué interpretación económica tienen las variables *r* y *w* encontradas.

## Librerías

In [51]:
import numpy as np
from scipy.optimize import root, brentq, brenth, ridder, bisect, newton, toms748 
import time
import warnings

## Parámetros y variables iniciales

In [52]:
A1, A2 = 1.2, 0.9
alpha1, alpha2 = 0.3, 0.6
Kbar, Lbar = 100.0, 50.0

# condición inicial
x0 = [50, 25, 0.5, 0.5]


## Definición del sistema

In [53]:
def F(x):
    K1, L1, r, w = x
    K2 = Kbar - K1
    L2 = Lbar - L1
    
    # ecuaciones de equilibrio
    f1 = r - A1*alpha1*(np.sign(K1) * (np.abs(K1))**(alpha1-1))*(np.sign(L1) * (np.abs(L1))**(1-alpha1))
    f2 = r - A2*alpha2*(K2**(alpha2-1))*(L2**(1-alpha2))
    f3 = w - 1*A1*(1-alpha1)*(np.sign(K1))*((np.abs(K1))**alpha1)*(L1**(-alpha1))
    f4 = w - A2*(1-alpha2)*(K2**alpha2)*(L2**(-alpha2))
    
    return [f1, f2, f3, f4]

## Lista de métodos

In [54]:
metodos = ['hybr', 'lm', 'broyden1', 'broyden2', 'anderson', 'linearmixing', 'diagbroyden', 'excitingmixing', 'krylov', 'df-sane']

## Evaluamos convergencia con los metodos de la funcion Root()

In [55]:
# Vamos imprimiendo los resultados en una tabla
print("Resultados de la Simulación con la función F original:")
print("-" * 110)
print(f"{'Método':<15} | {'Convergió':<12} | {'K1':<10} | {'L1':<10} | {'r':<10} | {'w':<10} | {'Tiempo (s)':<12} | {'Iteraciones'}")
print("-" * 110)

exitos = 0
for metodo in metodos:
    try:
        sol = root(F, x0, method=metodo)
        convergencia = "Sí" if sol.success else "No"
        
        # Comprobamos si la convergencia es exitosa Y si es la solución correcta
        if sol.success and np.isclose(sol.x[0], 56.28, atol=0.1):
            exitos += 1
            k1, l1, r, w = sol.x
            print(f"{metodo:<15} | {convergencia:<12} | {k1:<10.4f} | {l1:<10.4f} | {r:<10.4f} | {w:<10.4f} | {sol.nfev}")
        else:
            # Imprime "-" si no convergió o si convergió a una solución errónea
            print(f"{metodo:<15} | {convergencia:<12} | {'-':<10} | {'-':<10} | {'-':<10} | {'-':<10} | {sol.nfev}")
            
    except Exception as e:
        print(f"{metodo:<15} | {'Error':<12} | {'-':<10} | {'-':<10} | {'-':<10} | {'-':<10} | N/A")

print("-" * 80)
print(f"Total de convergencias exitosas para la simulación: {exitos} / {len(metodos)}")

Resultados de la Simulación con la función F original:
--------------------------------------------------------------------------------------------------------------
Método          | Convergió    | K1         | L1         | r          | w          | Tiempo (s)   | Iteraciones
--------------------------------------------------------------------------------------------------------------
hybr            | No           | -          | -          | -          | -          | 21
lm              | Sí           | 56.2834    | 40.9192    | 0.2880     | 0.9243     | 43
broyden1        | No           | -          | -          | -          | -          | 1002
broyden2        | No           | -          | -          | -          | -          | 1009
anderson        | Error        | -          | -          | -          | -          | N/A
linearmixing    | No           | -          | -          | -          | -          | 1002
diagbroyden     | Error        | -          | -          | -          | -   

Para las condiciones iniciales impuestas notamos que solo uno de los 10 métodos logra converger,además, obtenemos varias alertas/errores con ciertos métodos, esto en su mayoría se debe que al probar valores de K1 y L1 el método intenta calcular la raíz de un número negativo lo que da como resultado un número imaginario lo que arroja un NaN. Esto provoca que el array devuelto por la función contiene NaNs lo que "rompe" el código.

## Evaluación de Root() con función F retocada

Para evitar valores de K1 y L1 problemáticos, agregamos una condición dentro de la función. Esto garantiza que al intentar usar valores que rompen directamente el código se usan valores extremandamente alejados de 0 para que al utilizar los distintos métodos de root(), estos no sigan en esa dirección.

## Nueva función F

In [56]:
# Nueva funcion con if
def F_robusta(x):
    K1, L1, r, w = x
    
    # Si K1 o L1 se hacen negativos, los hacemos muy grandes para que el sistema busque en otra direccion
    if K1 <= 0 or L1 <= 0 or K1 >= Kbar or L1 >= Lbar:
        return [1e6, 1e6, 1e6, 1e6]

    K2 = Kbar - K1
    L2 = Lbar - L1
    
    f1 = r - A1 * alpha1 * (K1**(alpha1 - 1)) * (L1**(1 - alpha1))
    f2 = r - A2 * alpha2 * (K2**(alpha2 - 1)) * (L2**(1 - alpha2))
    f3 = w - A1 * (1 - alpha1) * (K1**alpha1) * (L1**(-alpha1))
    f4 = w - A2 * (1 - alpha2) * (K2**alpha2) * (L2**(-alpha2))
    
    return [f1, f2, f3, f4]


# Nueva simulación

In [57]:
# Vamos imprimiendo los resultados en una tabla
print("Resultados de la Simulación con la función F retocada:")
print("-" * 110)
print(f"{'Método':<15} | {'Convergió':<12} | {'K1':<10} | {'L1':<10} | {'r':<10} | {'w':<10} | {'Tiempo (s)':<12} | {'Iteraciones'}")
print("-" * 110)

exitos = 0
for metodo in metodos:
    try:
        sol = root(F_robusta, x0, method=metodo)
        convergencia = "Sí" if sol.success else "No"
        
        # Comprobamos si la convergencia es exitosa Y si es la solución correcta
        if sol.success and np.isclose(sol.x[0], 56.28, atol=0.1):
            exitos += 1
            k1, l1, r, w = sol.x
            print(f"{metodo:<15} | {convergencia:<12} | {k1:<10.4f} | {l1:<10.4f} | {r:<10.4f} | {w:<10.4f} | {sol.nfev}")
        else:
            # Imprime "-" si no convergió o si convergió a una solución errónea
            print(f"{metodo:<15} | {convergencia:<12} | {'-':<10} | {'-':<10} | {'-':<10} | {'-':<10} | {sol.nfev}")
            
    except Exception as e:
        print(f"{metodo:<15} | {'Error':<12} | {'-':<10} | {'-':<10} | {'-':<10} | {'-':<10} | N/A")

print("-" * 80)
print(f"Total de convergencias exitosas para la simulación: {exitos} / {len(metodos)}")

Resultados de la Simulación con la función F retocada:
--------------------------------------------------------------------------------------------------------------
Método          | Convergió    | K1         | L1         | r          | w          | Tiempo (s)   | Iteraciones
--------------------------------------------------------------------------------------------------------------
hybr            | No           | -          | -          | -          | -          | 21
lm              | Sí           | 56.2834    | 40.9192    | 0.2880     | 0.9243     | 53
broyden1        | Sí           | 56.2832    | 40.9191    | 0.2880     | 0.9243     | 68
broyden2        | No           | -          | -          | -          | -          | 1009
anderson        | Sí           | 56.2831    | 40.9191    | 0.2880     | 0.9243     | 313
linearmixing    | No           | -          | -          | -          | -          | 4181
diagbroyden     | No           | -          | -          | -          | -     

Con el arreglo planteado, pasamos de 1 a 3 convergencias y de 2 a 0 errores con los métodos, ahora bien, veamos si es posible hacer más de estos métodos converger.

## Variación de parámetros

Tratemos de lograr más convergencias haciendo variar los parámetros iniciales, primero, probemos con valores cercanos a los encontrados anteriormente

In [58]:
x1 = [60, 40, 0.3, 0.9]
nombre_x1 = "Valores cercanos"

print(f"\n=========================================================================")
print(f"  Probando con x0 = '{nombre_x1}'  (valores: {x1})")
print(f"=========================================================================")
print(f"{'Método':<15} | {'Convergió':<12} | {'K1':<10} | {'L1':<10} | {'r':<10} | {'w':<10} | {'Iteraciones'}")
print("-" * 80)

exitos = 0
for metodo in metodos:
    try:
        sol = root(F_robusta, x1, method=metodo)
        convergencia = "Sí" if sol.success else "No"
        
        # Comprobamos si la convergencia es exitosa Y si es la solución correcta
        if sol.success and np.isclose(sol.x[0], 56.28, atol=0.1):
            exitos += 1
            k1, l1, r, w = sol.x
            print(f"{metodo:<15} | {convergencia:<12} | {k1:<10.4f} | {l1:<10.4f} | {r:<10.4f} | {w:<10.4f} | {sol.nfev}")
        else:
            # Imprime "-" si no convergió o si convergió a una solución errónea
            print(f"{metodo:<15} | {convergencia:<12} | {'-':<10} | {'-':<10} | {'-':<10} | {'-':<10} | {sol.nfev}")
            
    except Exception as e:
        print(f"{metodo:<15} | {'Error':<12} | {'-':<10} | {'-':<10} | {'-':<10} | {'-':<10} | N/A")

print("-" * 80)
print(f"Total de convergencias exitosas para la simulación: {exitos} / {len(metodos)}")


  Probando con x0 = 'Valores cercanos'  (valores: [60, 40, 0.3, 0.9])
Método          | Convergió    | K1         | L1         | r          | w          | Iteraciones
--------------------------------------------------------------------------------
hybr            | Sí           | 56.2834    | 40.9192    | 0.2880     | 0.9243     | 14
lm              | Sí           | 56.2834    | 40.9192    | 0.2880     | 0.9243     | 28
broyden1        | No           | -          | -          | -          | -          | 1002
broyden2        | No           | -          | -          | -          | -          | 1001
anderson        | No           | -          | -          | -          | -          | 999
linearmixing    | No           | -          | -          | -          | -          | 4489
diagbroyden     | Error        | -          | -          | -          | -          | N/A
excitingmixing  | No           | -          | -          | -          | -          | 4486
krylov          | Sí           | 56.2

Observamos que pese a entregar valores cercanos a la solución, todavía logramos solo 3 convergencias y además en este caso obtenemos un error. Además, métodos que lograron converger con los parámetros originales ya no lo hacen. Esto se debe esencialmente a la manera en la que opera cada método además de la condición impuesta en la función F_robusta. Es como estar buscando el punto más bajo en un valle, los métodos calculan un step que significa que tan rápido avanzan en las direcciones que toman. El step es calculado por el Jacobiano y si la posición inicial dada es en plena "bajada" muy empinada el metodo decidirá dar steps muy grandes, saltandose los puntos más bajos o bien pasando a valores muy grandes que aplican la "seguridad" de la función F_robusta. En estos casos al entregar valores tan grandes ciertos métodos se "rompen" igualmente y se rinden.

Probemos entonces con valores muy cercanos a la raíz

In [59]:
x2 = [56.28, 40.91, 0.288, 0.924]
nombre_x2 = "Valores casi exactos"

print(f"\n=========================================================================")
print(f"  Probando con x0 = '{nombre_x2}'  (valores: {x2})")
print(f"=========================================================================")
print(f"{'Método':<15} | {'Convergió':<12} | {'K1':<10} | {'L1':<10} | {'r':<10} | {'w':<10} | {'Iteraciones'}")
print("-" * 80)

exitos = 0
for metodo in metodos:
    try:
        sol = root(F_robusta, x1, method=metodo)
        convergencia = "Sí" if sol.success else "No"
        
        # Comprobamos si la convergencia es exitosa Y si es la solución correcta
        if sol.success and np.isclose(sol.x[0], 56.28, atol=0.1):
            exitos += 1
            k1, l1, r, w = sol.x
            print(f"{metodo:<15} | {convergencia:<12} | {k1:<10.4f} | {l1:<10.4f} | {r:<10.4f} | {w:<10.4f} | {sol.nfev}")
        else:
            # Imprime "-" si no convergió o si convergió a una solución errónea
            print(f"{metodo:<15} | {convergencia:<12} | {'-':<10} | {'-':<10} | {'-':<10} | {'-':<10} | {sol.nfev}")
            
    except Exception as e:
        print(f"{metodo:<15} | {'Error':<12} | {'-':<10} | {'-':<10} | {'-':<10} | {'-':<10} | N/A")

print("-" * 80)
print(f"Total de convergencias exitosas para la simulación: {exitos} / {len(metodos)}")


  Probando con x0 = 'Valores casi exactos'  (valores: [56.28, 40.91, 0.288, 0.924])
Método          | Convergió    | K1         | L1         | r          | w          | Iteraciones
--------------------------------------------------------------------------------
hybr            | Sí           | 56.2834    | 40.9192    | 0.2880     | 0.9243     | 14
lm              | Sí           | 56.2834    | 40.9192    | 0.2880     | 0.9243     | 28
broyden1        | No           | -          | -          | -          | -          | 1002
broyden2        | No           | -          | -          | -          | -          | 1001
anderson        | No           | -          | -          | -          | -          | 999
linearmixing    | No           | -          | -          | -          | -          | 4489
diagbroyden     | Error        | -          | -          | -          | -          | N/A
excitingmixing  | No           | -          | -          | -          | -          | 4486
krylov          | Sí   

Observamos que incluso con valores casi iguales a los buscados, solo logramos convergencia con 3 metodos, veamos entonces a continuación si hay un punto ideal que asegure más convergencias.

In [64]:
# Silenciamos SOLAMENTE los RuntimeWarning para limpiar la salida
warnings.filterwarnings("ignore", category=RuntimeWarning)


# Creamos una parrilla de busqueda de puntos con r y w constantes
k1_grid = np.linspace(5, 95, 15)
l1_grid = np.linspace(5, 45, 15)
r0, w0 = 0.5, 0.5
resultados_grid = {}

print(f"--- Iniciando Búsqueda en Parrilla de {len(k1_grid) * len(l1_grid)} puntos x0 ---")
print(f"Probando {len(metodos)} métodos en cada punto... (Sin warnings)")

# Por cada punto encontrado, evaluamos y medimos convergencias
for k1 in k1_grid:
    for l1 in l1_grid:
        x0 = [k1, l1, r0, w0]
        exitos = 0
        
        for metodo in metodos:
            try:
                sol = root(F_robusta, x0, method=metodo)
                if sol.success and np.isclose(sol.x[0], 56.28, atol=0.1):
                    exitos += 1
            except Exception as e:
                continue 
        
        nombre_x0 = f"K1={k1:.1f}, L1={l1:.1f}"
        resultados_grid[nombre_x0] = exitos

print("--- Búsqueda Finalizada ---")

# Ordenamos los resultados y mostramos los 10 mejores
resultados_ordenados = sorted(resultados_grid.items(), key=lambda item: item[1], reverse=True)

print("\n--- Ranking de los 10 Mejores Puntos de Partida (x0) ---")
print("---------------------------------------------------------")
print(f"{'Punto de Partida (K1, L1)':<25} | {'Métodos Convergidos'}")
print("---------------------------------------------------------")

for (nombre, exitos) in resultados_ordenados[:10]:
    print(f"{nombre:<25} | {exitos} / {len(metodos)}")

# Reactivamos los warnings  
warnings.filterwarnings("default")

--- Iniciando Búsqueda en Parrilla de 225 puntos x0 ---
Probando 10 métodos en cada punto... (Sin warnings)
--- Búsqueda Finalizada ---

--- Ranking de los 10 Mejores Puntos de Partida (x0) ---
---------------------------------------------------------
Punto de Partida (K1, L1) | Métodos Convergidos
---------------------------------------------------------
K1=11.4, L1=19.3          | 5 / 10
K1=11.4, L1=25.0          | 5 / 10
K1=11.4, L1=27.9          | 5 / 10
K1=11.4, L1=30.7          | 5 / 10
K1=11.4, L1=33.6          | 5 / 10
K1=11.4, L1=39.3          | 5 / 10
K1=17.9, L1=25.0          | 5 / 10
K1=17.9, L1=27.9          | 5 / 10
K1=17.9, L1=30.7          | 5 / 10
K1=17.9, L1=36.4          | 5 / 10


Sería posible hacer una malla cada vez más grande para buscar un punto exacto en el que más métodos converjan pero queda claro que es casi imposible que esto suceda. Es muy dificil que logremos convergencia con linearmixing, diagbroyden y excitingmixing ya que son métodos antiguos y no adaptados a estas aplicaciones.